In [1]:
!pip install py-eureka-client

  Using cached py_eureka_client-0.11.13-py3-none-any.whl.metadata (19 kB)
  Using cached ifaddr-0.2.0-py3-none-any.whl.metadata (4.9 kB)
  Using cached dnspython-2.7.0-py3-none-any.whl.metadata (5.8 kB)
Using cached py_eureka_client-0.11.13-py3-none-any.whl (33 kB)
Using cached dnspython-2.7.0-py3-none-any.whl (313 kB)
Using cached ifaddr-0.2.0-py3-none-any.whl (12 kB)


In [9]:
pip install flask

  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
Using cached blinker-1.9.0-py3-none-any.whl (8.5 kB)
Using cached click-8.1.8-py3-none-any.whl (98 kB)
Note: you may need to restart the kernel to use updated packages.


In [23]:
import asyncio
import py_eureka_client.eureka_client as eureka_client

async def register_to_eureka():
    await eureka_client.stop_async()  # Por si ya hay una instancia corriendo
    await eureka_client.init_async(
        eureka_server="http://localhost:8099/eureka,http://172.29.88.29:8099/eureka",
        app_name="Alumnos",
        instance_port=8099
    )

await register_to_eureka()


[2025-04-17 22:34:07]-[MainThread]-[eureka_client:1107] WARNING: A client is already running, try to stop it and start the new one!


In [25]:
pip install fastapi uvicorn

  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 46.5 MB/s eta 0:00:00
Using cached idna-3.10-py3-none-any.whl (70 kB)
Note: you may need to restart the kernel to use updated packages.


In [26]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel

In [ ]:
app = FastAPI()


class Alumno(BaseModel):
    nombre: str
    correo: str
    contrasena: str

alumnos_registrados = []

@app.post("/registrar")
def registrar_alumno(alumno: Alumno):
    if not alumno.nombre or not alumno.correo or not alumno.contrasena:
        raise HTTPException(status_code=400, detail="Todos los campos son obligatorios")

    alumno_data = {
        "id": len(alumnos_registrados) + 1,
        "nombre": alumno.nombre,
        "correo": alumno.correo,
        "contrasena": alumno.contrasena  
    }

    alumnos_registrados.append(alumno_data)
    return {"mensaje": "Alumno registrado exitosamente", "alumno": alumno_data}

In [ ]:
def login_alumno(alumno: Alumno):
   
    alumno_encontrado = next((a for a in alumnos_registrados if a["correo"] == alumno.correo), None)

    if alumno_encontrado is None:
        raise HTTPException(status_code=404, detail="Alumno no encontrado")

    if alumno_encontrado["contrasena"] != alumno.contrasena:
        raise HTTPException(status_code=401, detail="Contraseña incorrecta")
    return {"mensaje": "Login exitoso", "alumno": alumno_encontrado}